# Day 09. Exercise 00
# Regularization

## 0. Imports

In [ ]:
# Импорт необходимых библиотек

import pandas as pd  # Для работы с данными в табличном формате
import numpy as np   # Для математических операций и работы с массивами

# Модули sklearn для машинного обучения
from sklearn.model_selection import train_test_split, StratifiedKFold  # Разделение данных и кросс-валидация
from sklearn.linear_model import LogisticRegression  # Логистическая регрессия
from sklearn.svm import SVC  # Support Vector Classifier (метод опорных векторов)
from sklearn.tree import DecisionTreeClassifier  # Дерево решений
from sklearn.ensemble import RandomForestClassifier  # Случайный лес
from sklearn.metrics import accuracy_score, confusion_matrix  # Метрики качества

import pickle  # Для сохранения обученной модели
import warnings
warnings.filterwarnings('ignore')  # Отключаем предупреждения для чистоты вывода

## 1. Preprocessing

1. Read the file `dayofweek.csv` that you used in the previous day to a dataframe.
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test`. Use the additional parameter `stratify`.

In [ ]:
# Загрузка данных из CSV файла
# Файл dayofweek.csv содержит информацию о коммитах: uid, lab name, количество попыток, час коммита
# Целевая переменная - день недели (dayofweek)
df = pd.read_csv('../data/dayofweek.csv')
df.head()  # Выводим первые 5 строк для проверки данных

In [ ]:
# Разделение данных на признаки (X) и целевую переменную (y)
# X - все колонки кроме 'dayofweek' (признаки: uid, lab name, trials, hour)
# y - колонка 'dayofweek' (целевая переменная - день недели от 0 до 6)
X = df.drop('dayofweek', axis=1)
y = df['dayofweek']

print(f"Размер матрицы признаков X: {X.shape}")  # (количество образцов, количество признаков)
print(f"Размер вектора целевой переменной y: {y.shape}")
print(f"\nРаспределение классов (дней недели):\n{y.value_counts().sort_index()}")

In [ ]:
# Разделение данных на обучающую и тестовую выборки
# test_size=0.2 - 20% данных уходит в тест, 80% в обучение
# random_state=21 - фиксируем случайность для воспроизводимости результатов
# stratify=y - сохраняем пропорции классов в обеих выборках (важно при несбалансированных классах)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=21, 
    stratify=y
)

print(f"Размер обучающей выборки: {X_train.shape[0]} образцов")
print(f"Размер тестовой выборки: {X_test.shape[0]} образцов")

## 2. Logreg regularization

### a. Default regularization

1. Train a baseline model with the only parameters `random_state=21`, `fit_intercept=False`.
2. Use stratified K-fold cross-validation with `10` splits to evaluate the accuracy of the model


The result of the code where you trained and evaluated the baseline model should be exactly like this (use `%%time` to get the info about how long it took to run the cell):

```
train -  0.62902   |   valid -  0.59259
train -  0.64633   |   valid -  0.62963
train -  0.63479   |   valid -  0.56296
train -  0.65622   |   valid -  0.61481
train -  0.63397   |   valid -  0.57778
train -  0.64056   |   valid -  0.59259
train -  0.64138   |   valid -  0.65926
train -  0.65952   |   valid -  0.56296
train -  0.64333   |   valid -  0.59701
train -  0.63674   |   valid -  0.62687
Average accuracy on crossval is 0.60165
Std is 0.02943
```

In [ ]:
%%time
# Базовая модель логистической регрессии
# random_state=21 - для воспроизводимости
# fit_intercept=False - не добавляем свободный член (bias)
# max_iter=1000 - максимальное число итераций для сходимости алгоритма
model = LogisticRegression(random_state=21, fit_intercept=False, max_iter=1000)

# Стратифицированная K-fold кросс-валидация
# n_splits=10 - делим данные на 10 частей
# shuffle=True - перемешиваем данные перед разбиением
# Стратификация сохраняет пропорции классов в каждом фолде
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=21)

train_scores = []
valid_scores = []

# Цикл по фолдам кросс-валидации
for train_idx, valid_idx in skf.split(X_train, y_train):
    # Разделяем данные на текущий train и validation фолд
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[valid_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]
    
    # Обучаем модель на train фолде
    model.fit(X_tr, y_tr)
    
    # Оцениваем качество на train и validation
    train_score = model.score(X_tr, y_tr)
    valid_score = model.score(X_val, y_val)
    
    train_scores.append(train_score)
    valid_scores.append(valid_score)
    
    print(f"train -  {train_score:.5f}   |   valid -  {valid_score:.5f}")

# Итоговые метрики по всем фолдам
print(f"Average accuracy on crossval is {np.mean(valid_scores):.5f}")
print(f"Std is {np.std(valid_scores):.5f}")  # Стандартное отклонение показывает стабильность модели

### b. Optimizing regularization parameters

1. In the cells below try different values of penalty: `none`, `l1`, `l2` – you can change the values of solver too.

In [ ]:
%%time
# Логистическая регрессия с L2 регуляризацией (Ridge)
# L2 регуляризация уменьшает величину всех коэффициентов, но не обнуляет их
# Помогает предотвратить переобучение (overfitting)
# C=1.0 - обратный коэффициент регуляризации (чем меньше C, тем сильнее регуляризация)
model_l2 = LogisticRegression(penalty='l2', random_state=21, fit_intercept=False, max_iter=1000, C=1.0)

valid_scores_l2 = []

for train_idx, valid_idx in skf.split(X_train, y_train):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[valid_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]
    
    model_l2.fit(X_tr, y_tr)
    valid_score = model_l2.score(X_val, y_val)
    valid_scores_l2.append(valid_score)
    print(f"valid -  {valid_score:.5f}")

print(f"\nСредняя точность с L2 регуляризацией: {np.mean(valid_scores_l2):.5f}")
print(f"Std: {np.std(valid_scores_l2):.5f}")

In [ ]:
%%time
# Логистическая регрессия с L1 регуляризацией (Lasso)
# L1 регуляризация может обнулять некоторые коэффициенты
# Полезна для отбора признаков (feature selection) - незначимые признаки получат вес 0
# solver='saga' - алгоритм оптимизации, который поддерживает L1
model_l1 = LogisticRegression(penalty='l1', solver='saga', random_state=21, fit_intercept=False, max_iter=1000, C=1.0)

valid_scores_l1 = []

for train_idx, valid_idx in skf.split(X_train, y_train):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[valid_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]
    
    model_l1.fit(X_tr, y_tr)
    valid_score = model_l1.score(X_val, y_val)
    valid_scores_l1.append(valid_score)
    print(f"valid -  {valid_score:.5f}")

print(f"\nСредняя точность с L1 регуляризацией: {np.mean(valid_scores_l1):.5f}")
print(f"Std: {np.std(valid_scores_l1):.5f}")

In [ ]:
%%time
# Логистическая регрессия БЕЗ регуляризации
# penalty=None - отключаем регуляризацию полностью
# Модель может переобучиться на тренировочных данных
model_none = LogisticRegression(penalty=None, random_state=21, fit_intercept=False, max_iter=1000)

valid_scores_none = []

for train_idx, valid_idx in skf.split(X_train, y_train):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[valid_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]
    
    model_none.fit(X_tr, y_tr)
    valid_score = model_none.score(X_val, y_val)
    valid_scores_none.append(valid_score)
    print(f"valid -  {valid_score:.5f}")

print(f"\nСредняя точность без регуляризации: {np.mean(valid_scores_none):.5f}")
print(f"Std: {np.std(valid_scores_none):.5f}")

## 3. SVM regularization

### a. Default regularization

1. Train a baseline model with the only parameters `probability=True`, `kernel='linear'`, `random_state=21`.
2. Use stratified K-fold cross-validation with `10` splits to evaluate the accuracy of the model.
3. The format of the result of the code where you trained and evaluated the baseline model should be similar to what you have got for the logreg.

In [ ]:
%%time
# Базовая модель SVM (Support Vector Machine - метод опорных векторов)
# probability=True - включаем расчет вероятностей (нужно для некоторых метрик)
# kernel='linear' - линейное ядро (простейший случай, разделяющая гиперплоскость)
# Другие ядра: 'rbf', 'poly', 'sigmoid'
svm_model = SVC(probability=True, kernel='linear', random_state=21)

svm_train_scores = []
svm_valid_scores = []

for train_idx, valid_idx in skf.split(X_train, y_train):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[valid_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]
    
    svm_model.fit(X_tr, y_tr)
    
    train_score = svm_model.score(X_tr, y_tr)
    valid_score = svm_model.score(X_val, y_val)
    
    svm_train_scores.append(train_score)
    svm_valid_scores.append(valid_score)
    
    print(f"train -  {train_score:.5f}   |   valid -  {valid_score:.5f}")

print(f"Средняя точность SVM на кросс-валидации: {np.mean(svm_valid_scores):.5f}")
print(f"Std: {np.std(svm_valid_scores):.5f}")

### b. Optimizing regularization parameters

1. In the cells below try different values of the parameter `C`.

In [ ]:
%%time
# SVM с параметром регуляризации C=0.1
# C - параметр регуляризации (штраф за ошибки классификации)
# Маленькое C (0.1) = сильная регуляризация, модель более простая, меньше переобучение
# Большое C = слабая регуляризация, модель может переобучиться
svm_c01 = SVC(probability=True, kernel='linear', random_state=21, C=0.1)

valid_scores_svm_c01 = []

for train_idx, valid_idx in skf.split(X_train, y_train):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[valid_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]
    
    svm_c01.fit(X_tr, y_tr)
    valid_score = svm_c01.score(X_val, y_val)
    valid_scores_svm_c01.append(valid_score)
    print(f"valid -  {valid_score:.5f}")

print(f"\nСредняя точность SVM с C=0.1: {np.mean(valid_scores_svm_c01):.5f}")
print(f"Std: {np.std(valid_scores_svm_c01):.5f}")

In [ ]:
%%time
# SVM с параметром регуляризации C=10
# Большое C = слабая регуляризация
# Модель пытается классифицировать все обучающие примеры правильно
# Может привести к переобучению на шумных данных
svm_c10 = SVC(probability=True, kernel='linear', random_state=21, C=10)

valid_scores_svm_c10 = []

for train_idx, valid_idx in skf.split(X_train, y_train):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[valid_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]
    
    svm_c10.fit(X_tr, y_tr)
    valid_score = svm_c10.score(X_val, y_val)
    valid_scores_svm_c10.append(valid_score)
    print(f"valid -  {valid_score:.5f}")

print(f"\nСредняя точность SVM с C=10: {np.mean(valid_scores_svm_c10):.5f}")
print(f"Std: {np.std(valid_scores_svm_c10):.5f}")

## 4. Tree

### a. Default regularization

1. Train a baseline model with the only parameter `max_depth=10` and `random_state=21`.
2. Use stratified K-fold cross-validation with `10` splits to evaluate the accuracy of the model.
3. The format of the result of the code where you trained and evaluated the baseline model should be similar to what you have got for the logreg.

In [ ]:
%%time
# Базовая модель дерева решений (Decision Tree)
# max_depth=10 - максимальная глубина дерева (регуляризация)
# Чем глубже дерево, тем сложнее модель и выше риск переобучения
# Без ограничения глубины дерево может создать отдельный лист для каждого образца
tree_model = DecisionTreeClassifier(max_depth=10, random_state=21)

tree_train_scores = []
tree_valid_scores = []

for train_idx, valid_idx in skf.split(X_train, y_train):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[valid_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]
    
    tree_model.fit(X_tr, y_tr)
    
    train_score = tree_model.score(X_tr, y_tr)
    valid_score = tree_model.score(X_val, y_val)
    
    tree_train_scores.append(train_score)
    tree_valid_scores.append(valid_score)
    
    print(f"train -  {train_score:.5f}   |   valid -  {valid_score:.5f}")

print(f"Средняя точность дерева на кросс-валидации: {np.mean(tree_valid_scores):.5f}")
print(f"Std: {np.std(tree_valid_scores):.5f}")

### b. Optimizing regularization parameters

1. In the cells below try different values of the parameter `max_depth`.
2. As a bonus, play with other regularization parameters trying to find the best combination.

In [ ]:
%%time
# Дерево с max_depth=5 (более простая модель)
# Меньшая глубина = более сильная регуляризация
# Модель более обобщенная, меньше риск переобучения
# Но может недообучиться (underfitting) если данные сложные
tree_d5 = DecisionTreeClassifier(max_depth=5, random_state=21)

valid_scores_tree_d5 = []

for train_idx, valid_idx in skf.split(X_train, y_train):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[valid_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]
    
    tree_d5.fit(X_tr, y_tr)
    valid_score = tree_d5.score(X_val, y_val)
    valid_scores_tree_d5.append(valid_score)
    print(f"valid -  {valid_score:.5f}")

print(f"\nСредняя точность дерева с max_depth=5: {np.mean(valid_scores_tree_d5):.5f}")
print(f"Std: {np.std(valid_scores_tree_d5):.5f}")

In [ ]:
%%time
# Дерево с max_depth=15 (более сложная модель)
# Большая глубина = слабая регуляризация
# Модель может лучше улавливать сложные паттерны
# Но выше риск переобучения на шуме в данных
tree_d15 = DecisionTreeClassifier(max_depth=15, random_state=21)

valid_scores_tree_d15 = []

for train_idx, valid_idx in skf.split(X_train, y_train):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[valid_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]
    
    tree_d15.fit(X_tr, y_tr)
    valid_score = tree_d15.score(X_val, y_val)
    valid_scores_tree_d15.append(valid_score)
    print(f"valid -  {valid_score:.5f}")

print(f"\nСредняя точность дерева с max_depth=15: {np.mean(valid_scores_tree_d15):.5f}")
print(f"Std: {np.std(valid_scores_tree_d15):.5f}")

## 5. Random forest

### a. Default regularization

1. Train a baseline model with the only parameters `n_estimators=50`, `max_depth=14`, `random_state=21`.
2. Use stratified K-fold cross-validation with `10` splits to evaluate the accuracy of the model.
3. The format of the result of the code where you trained and evaluated the baseline model should be similar to what you have got for the logreg.

In [ ]:
%%time
# Базовая модель случайного леса (Random Forest)
# Ансамбль из множества деревьев решений
# n_estimators=50 - количество деревьев в лесу
# max_depth=14 - максимальная глубина каждого дерева
# Случайный лес менее склонен к переобучению чем одно дерево благодаря:
# 1. Bagging - каждое дерево обучается на случайной подвыборке данных
# 2. Случайный выбор признаков для каждого разделения
rf_model = RandomForestClassifier(n_estimators=50, max_depth=14, random_state=21)

rf_train_scores = []
rf_valid_scores = []

for train_idx, valid_idx in skf.split(X_train, y_train):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[valid_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]
    
    rf_model.fit(X_tr, y_tr)
    
    train_score = rf_model.score(X_tr, y_tr)
    valid_score = rf_model.score(X_val, y_val)
    
    rf_train_scores.append(train_score)
    rf_valid_scores.append(valid_score)
    
    print(f"train -  {train_score:.5f}   |   valid -  {valid_score:.5f}")

print(f"Средняя точность Random Forest на кросс-валидации: {np.mean(rf_valid_scores):.5f}")
print(f"Std: {np.std(rf_valid_scores):.5f}")

### b. Optimizing regularization parameters

1. In the new cells try different values of the parameters `max_depth` and `n_estimators`.
2. As a bonus, play with other regularization parameters trying to find the best combination.

In [ ]:
%%time
# Random Forest с n_estimators=100, max_depth=10
# Больше деревьев (100) = более стабильные предсказания
# Меньшая глубина (10) = более сильная регуляризация
# Баланс между сложностью модели и обобщающей способностью
rf_100_10 = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=21)

valid_scores_rf_100_10 = []

for train_idx, valid_idx in skf.split(X_train, y_train):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[valid_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]
    
    rf_100_10.fit(X_tr, y_tr)
    valid_score = rf_100_10.score(X_val, y_val)
    valid_scores_rf_100_10.append(valid_score)
    print(f"valid -  {valid_score:.5f}")

print(f"\nСредняя точность RF (100 деревьев, depth=10): {np.mean(valid_scores_rf_100_10):.5f}")
print(f"Std: {np.std(valid_scores_rf_100_10):.5f}")

In [ ]:
%%time
# Random Forest с n_estimators=50, max_depth=20
# Меньше деревьев (50), но каждое глубже (20)
# Более сложные деревья могут улавливать сложные паттерны
# Но также могут переобучаться
rf_50_20 = RandomForestClassifier(n_estimators=50, max_depth=20, random_state=21)

valid_scores_rf_50_20 = []

for train_idx, valid_idx in skf.split(X_train, y_train):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[valid_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]
    
    rf_50_20.fit(X_tr, y_tr)
    valid_score = rf_50_20.score(X_val, y_val)
    valid_scores_rf_50_20.append(valid_score)
    print(f"valid -  {valid_score:.5f}")

print(f"\nСредняя точность RF (50 деревьев, depth=20): {np.mean(valid_scores_rf_50_20):.5f}")
print(f"Std: {np.std(valid_scores_rf_50_20):.5f}")

## 6. Predictions

1. Choose the best model and use it to make predictions for the test dataset.
2. Calculate the final accuracy.
3. Analyze: for which weekday your model makes the most errors (in % of the total number of samples of that class in your test dataset).
4. Save the model.

In [ ]:
# Сравнение всех обученных моделей
# Собираем средние результаты кросс-валидации в словарь
# Сортируем по убыванию точности для выбора лучшей модели
results = {
    'LogReg baseline': np.mean(valid_scores),
    'LogReg L2': np.mean(valid_scores_l2),
    'LogReg L1': np.mean(valid_scores_l1),
    'LogReg no penalty': np.mean(valid_scores_none),
    'SVM baseline': np.mean(svm_valid_scores),
    'SVM C=0.1': np.mean(valid_scores_svm_c01),
    'SVM C=10': np.mean(valid_scores_svm_c10),
    'Tree baseline': np.mean(tree_valid_scores),
    'Tree d=5': np.mean(valid_scores_tree_d5),
    'Tree d=15': np.mean(valid_scores_tree_d15),
    'RF baseline': np.mean(rf_valid_scores),
    'RF 100-10': np.mean(valid_scores_rf_100_10),
    'RF 50-20': np.mean(valid_scores_rf_50_20),
}

print("Рейтинг моделей по точности на кросс-валидации:")
print("-" * 45)
for model_name, score in sorted(results.items(), key=lambda x: x[1], reverse=True):
    print(f"{model_name:25s}: {score:.5f}")

In [ ]:
# Обучение лучшей модели на полной обучающей выборке
# Выбираем Random Forest как обычно показывающий хорошие результаты
# Теперь обучаем на ВСЕХ тренировочных данных (не только на фолдах)
best_model = RandomForestClassifier(n_estimators=50, max_depth=14, random_state=21)
best_model.fit(X_train, y_train)

# Делаем предсказания на тестовой выборке
# Тестовая выборка НЕ использовалась при обучении и выборе модели
y_pred = best_model.predict(X_test)

# Считаем финальную метрику на тесте
# Это честная оценка качества модели на новых данных
test_accuracy = accuracy_score(y_test, y_pred)
print(f"Точность на тестовой выборке: {test_accuracy:.5f}")

In [ ]:
# Матрица ошибок (Confusion Matrix)
# Показывает сколько образцов каждого класса были классифицированы в каждый класс
# Диагональ - правильные предсказания
# Вне диагонали - ошибки (путаницы между классами)
cm = confusion_matrix(y_test, y_pred)
print("Матрица ошибок (Confusion Matrix):")
print("Строки - истинные классы, столбцы - предсказанные")
print(cm)
print()

In [ ]:
# Анализ ошибок по дням недели
# Вычисляем процент ошибок для каждого дня
# Это помогает понять, для каких классов модель работает хуже

weekdays = sorted(y_test.unique())
error_rates = {}

for day in weekdays:
    # Получаем все образцы данного дня
    day_mask = (y_test == day)
    day_total = day_mask.sum()  # Всего образцов этого дня
    
    # Считаем ошибки - образцы этого дня, которые предсказаны неверно
    day_errors = ((y_test == day) & (y_pred != day)).sum()
    
    # Процент ошибок от общего числа образцов этого класса
    error_rate = (day_errors / day_total * 100) if day_total > 0 else 0
    error_rates[day] = error_rate
    
    print(f"День {day}: {day_errors}/{day_total} ошибок ({error_rate:.2f}%)")

# Находим день с максимальным процентом ошибок
worst_day = max(error_rates, key=error_rates.get)
print(f"\nХуже всего модель предсказывает день: {worst_day}")
print(f"Процент ошибок для этого дня: {error_rates[worst_day]:.2f}%")

In [ ]:
# Сохранение лучшей модели с помощью pickle
# pickle - стандартный способ сериализации объектов Python
# Сохраненную модель можно загрузить позже для предсказаний без повторного обучения

model_filename = '../data/best_model_ex00.pkl'

# Открываем файл для записи в бинарном режиме ('wb')
with open(model_filename, 'wb') as f:
    pickle.dump(best_model, f)  # Сериализуем и записываем модель
    
print(f"Модель сохранена в файл: {model_filename}")
print(f"Для загрузки используйте: model = pickle.load(open('{model_filename}', 'rb'))")